# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
pip install citipy

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "../output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [3]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

623

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [4]:
# Base url
url = f"http://api.openweathermap.org/data/2.5/weather?units=imperial&appid={weather_api_key}&q="

# List of city data
cityData = []

# Grouping variables
# Group
set_count = 1
# Record within group
record = 1

print("Beginning Data Retrieval")
print("--------------------------------")

for i, city in enumerate(cities):
    # group the cities in sets of 50
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record = 1
        
    # Output the record and group being processed
    print(f"Processing Record {record} of Set {set_count} | {city}")
    
    # Increase record number in the set
    record += 1
    
    # Create URL that includes city
    cityURL = url + city
    
    # Call the OpenWeather API
    try:
        cityWeather = requests.get(cityURL).json()
        
        # Parse out data
        lat = cityWeather["coord"]["lat"]
        lng = cityWeather["coord"]["lon"]
        maxTemp = cityWeather["main"]["temp_max"]
        humidity = cityWeather["main"]["humidity"]
        cloudiness = cityWeather["clouds"]["all"]
        windspeed = cityWeather["wind"]["speed"]
        country = cityWeather["sys"]["country"]
        date = cityWeather["dt"]
        
        cityData.append({"City": city,
                        "Lat": lat,
                        "Lng": lng,
                        "Max Temp": maxTemp,
                        "Humidity": humidity,
                        "Cloudiness": cloudiness,
                        "Wind Speed": windspeed,
                         "Country": country, 
                        "Date": date})
    except:
        print(f"{city} not found. Skipping...")
        pass
    
    if i == 5:
        break

Beginning Data Retrieval
--------------------------------
Processing Record 1 of Set 1 | luangwa
Processing Record 2 of Set 1 | iqaluit
Processing Record 3 of Set 1 | bredasdorp
Processing Record 4 of Set 1 | lucea
Processing Record 5 of Set 1 | barrow
Processing Record 6 of Set 1 | severo-kurilsk


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [5]:
# Creates a data frame out of the raw data, and exports it as a csv
cityData_df = pd.DataFrame(cityData)
cityData_df.to_csv(output_data_file)

In [6]:
# Displays the data frame
cityData_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,luangwa,-15.6167,30.4167,65.35,81,1,2.66,ZM,1619133165
1,iqaluit,63.7506,-68.5145,21.20,86,90,3.44,CA,1619132953
2,bredasdorp,-34.5322,20.0403,62.08,92,1,1.88,ZA,1619133166
3,lucea,18.4510,-78.1736,84.00,78,75,10.36,JM,1619133166
4,barrow,71.2906,-156.7887,21.20,86,1,8.05,US,1619133166
5,severo-kurilsk,50.6789,156.1250,35.71,93,69,3.11,RU,1619133166


In [7]:
cityData_df.describe()

,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
count,6.000000,6.000000,6.000000,6.000000,6.00000,6.000000,6.000000e+00
mean,25.670367,-16.149133,48.256667,86.000000,39.50000,4.916667,1.619133e+09
std,43.675684,109.110112,26.009220,5.899152,42.72587,3.441108,8.687616e+01
min,-34.532200,-156.788700,21.200000,78.000000,1.00000,1.880000,1.619133e+09
25%,-7.099775,-75.758825,24.827500,82.250000,1.00000,2.772500,1.619133e+09
50%,34.564950,-24.237100,48.895000,86.000000,35.00000,3.275000,1.619133e+09
75%,60.482675,27.822600,64.532500,90.500000,73.50000,6.897500,1.619133e+09
max,71.290600,156.125000,84.000000,93.000000,90.00000,10.360000,1.619133e+09


## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [29]:
#  Get the indices of cities that have humidity over 100%.
index_humidity = cityData_df.index[cityData_df["Humidity"] >= 100].tolist()
index_humidity

[]

In [31]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".
clean_city_data = cityData_df.drop(index_humidity)
clean_city_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,luangwa,-15.6167,30.4167,65.35,81,1,2.66,ZM,1619133165
1,iqaluit,63.7506,-68.5145,21.20,86,90,3.44,CA,1619132953
2,bredasdorp,-34.5322,20.0403,62.08,92,1,1.88,ZA,1619133166
3,lucea,18.4510,-78.1736,84.00,78,75,10.36,JM,1619133166
4,barrow,71.2906,-156.7887,21.20,86,1,8.05,US,1619133166


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression